The [data_algebra](https://github.com/WinVector/data_algebra) provides a simplified (though verbose) unified interface to Pandas and SQL windows functions.

Let's work an example. First bring in our packages.

In [1]:
import sqlite3
import pandas

from data_algebra.data_ops import *  # https://github.com/WinVector/data_algebra
import data_algebra.util
import data_algebra.SQLite

Now some example data.

In [2]:
d = pandas.DataFrame({
    'g': [1, 2, 2, 3, 3, 3],
    'x': [1, 4, 5, 7, 8, 9],
    'v': [10, 40, 50, 70, 80, 90],
})

And we can run a number of ordered and un-ordered window functions (the distinction is given by if ther is an `order_by` argument present).

In [3]:
table_desciption = describe_table(d)

ops = table_desciption. \
    extend({
        'row_number': '_row_number()',
        'shift_v': 'v.shift()',
    },
    order_by=['x'],
    partition_by=['g']). \
    extend({
        'ngroup': '_ngroup()',
        'size': '_size()',
        'max_v': 'v.max()',
        'min_v': 'v.min()',
        'sum_v': 'v.sum()',
        'mean_v': 'v.mean()',
        'count_v': 'v.count()',
        'size_v': 'v.size()',
    },
    partition_by=['g'])

res1 = ops.transform(d)

res1

,g,x,v,row_number,shift_v,ngroup,size,max_v,min_v,sum_v,mean_v,count_v,size_v
0,1,1,10,1,NaN,0,1,10,10,10,10,1,1
1,2,4,40,1,NaN,1,2,50,40,90,45,2,2
2,2,5,50,2,40.0,1,2,50,40,90,45,2,2
3,3,7,70,1,NaN,2,3,90,70,240,80,3,3
4,3,8,80,2,70.0,2,3,90,70,240,80,3,3
5,3,9,90,3,80.0,2,3,90,70,240,80,3,3


One of the benefits of the data_algebra is the commands are saved in an object.

In [4]:
print(ops.to_python(pretty=True))

TableDescription(table_name="data_frame", column_names=["g", "x", "v"]
).extend(
    {"row_number": "_row_number()", "shift_v": "v.shift()"},
    partition_by=["g"],
    order_by=["x"],
).extend(
    {
        "ngroup": "_ngroup()",
        "size": "_size()",
        "max_v": "v.max()",
        "min_v": "v.min()",
        "sum_v": "v.sum()",
        "mean_v": "v.mean()",
        "count_v": "v.count()",
        "size_v": "v.size()",
    },
    partition_by=["g"],
)



And thiese commands can be re-used and even exported to SQL (including large scale SQL such as PostgreSQL, Apache Spark, or Google Big Query).

For a simple demonstration we will use small-scale SQL as realized in SQLite.

In [5]:
conn = sqlite3.connect(":memory:")
db_model = data_algebra.SQLite.SQLiteModel()
db_model.prepare_connection(conn)

ops_db = table_desciption. \
    extend({
        'row_number': '_row_number()',
        'shift_v': 'v.shift()',
    },
    order_by=['x'],
    partition_by=['g']). \
    extend({
        # 'ngroup': '_ngroup()',
        'size': '_size()',
        'max_v': 'v.max()',
        'min_v': 'v.min()',
        'sum_v': 'v.sum()',
        'mean_v': 'v.mean()',
        'count_v': 'v.count()',
        'size_v': 'v.size()',
    },
    partition_by=['g'])

db_model.insert_table(conn, d, table_desciption.table_name)
sql1 = ops_db.to_sql(db_model, pretty=True)

print(sql1)

SELECT "g",
       "x",
       "v",
       "row_number",
       "shift_v",
       SUM(1) OVER (PARTITION BY "g") AS "size",
                   MAX("v") OVER (PARTITION BY "g") AS "max_v",
                                 MIN("v") OVER (PARTITION BY "g") AS "min_v",
                                               SUM("v") OVER (PARTITION BY "g") AS "sum_v",
                                                             avg("v") OVER (PARTITION BY "g") AS "mean_v",
                                                                           COUNT("v") OVER (PARTITION BY "g") AS "count_v",
                                                                                           SUM(1) OVER (PARTITION BY "g") AS "size_v"
FROM
  (SELECT "g",
          "x",
          "v",
          ROW_NUMBER() OVER (PARTITION BY "g"
                             ORDER BY "x") AS "row_number",
                            LAG("v") OVER (PARTITION BY "g"
                                           ORDER BY "x") AS "

And we can execute this SQL either to materialize a remote result (which involves no data motion, as we send the SQL commands to the database, not move the data to/from Python), or to bring a result back from the database to Python.

In [6]:
res1_db = db_model.read_query(conn, sql1)

res1_db

,g,x,v,row_number,shift_v,size,max_v,min_v,sum_v,mean_v,count_v,size_v
0,1,1,10,1,NaN,1,10,10,10,10.0,1,1
1,2,4,40,1,NaN,2,50,40,90,45.0,2,2
2,2,5,50,2,40.0,2,50,40,90,45.0,2,2
3,3,7,70,1,NaN,3,90,70,240,80.0,3,3
4,3,8,80,2,70.0,3,90,70,240,80.0,3,3
5,3,9,90,3,80.0,3,90,70,240,80.0,3,3


Notice we didn't calculate the group-id `rgroup` in the `SQL` version.  This is because this is a much less common window function (and not often used in applications). This is also only interesting when we are using a composite key (else the single key column is already the per-group id). So not all data_algebra pipelines can run in all environments. However, we can compute (arbitrary) group IDs in a domain independent manner as follows.

In [7]:
id_ops_a = table_desciption. \
    project(group_by=['g']). \
    extend({
        'ngroup': '_row_number()',
    },
    order_by=['g'])

id_ops_b = table_desciption. \
    natural_join(id_ops_a, by=['g'], jointype='LEFT')

print(id_ops_b.to_python(pretty=True))

TableDescription(table_name="data_frame", column_names=["g", "x", "v"]
).natural_join(
    b=TableDescription(table_name="data_frame", column_names=["g", "x", "v"])
    .project({}, group_by=["g"])
    .extend({"ngroup": "_row_number()"}, order_by=["g"]),
    by=["g"],
    jointype="LEFT",
)



In [8]:
sql2 = id_ops_b.to_sql(db_model)

cur = conn.cursor()
cur.execute('CREATE TABLE remote_result AS ' + sql2)

In [9]:
res2_db = db_model.read_table(conn, 'remote_result')

res2_db

,g,v,x,ngroup
0,1,10,1,1
1,2,40,4,2
2,2,50,5,2
3,3,70,7,3
4,3,80,8,3
5,3,90,9,3


In [10]:
# clean up
conn.close()